Extracción de datos GEE

In [ ]:
import ee 
import geemap
import geopandas
import json
import os

In [ ]:
ee.Authenticate()
ee.Initialize(project='ee-rimartinezs')



image =  ee.Image('LANDSAT/LC08/C02/T1_L2/LC08_008057_20241011')
image.getInfo() 


{'type': 'Image',
 'bands': [{'id': 'SR_B1',
   'data_type': {'type': 'PixelType',
    'precision': 'int',
    'min': 0,
    'max': 65535},
   'dimensions': [7581, 7731],
   'crs': 'EPSG:32618',
   'crs_transform': [30, 0, 443985, 0, -30, 595215]},
  {'id': 'SR_B2',
   'data_type': {'type': 'PixelType',
    'precision': 'int',
    'min': 0,
    'max': 65535},
   'dimensions': [7581, 7731],
   'crs': 'EPSG:32618',
   'crs_transform': [30, 0, 443985, 0, -30, 595215]},
  {'id': 'SR_B3',
   'data_type': {'type': 'PixelType',
    'precision': 'int',
    'min': 0,
    'max': 65535},
   'dimensions': [7581, 7731],
   'crs': 'EPSG:32618',
   'crs_transform': [30, 0, 443985, 0, -30, 595215]},
  {'id': 'SR_B4',
   'data_type': {'type': 'PixelType',
    'precision': 'int',
    'min': 0,
    'max': 65535},
   'dimensions': [7581, 7731],
   'crs': 'EPSG:32618',
   'crs_transform': [30, 0, 443985, 0, -30, 595215]},
  {'id': 'SR_B5',
   'data_type': {'type': 'PixelType',
    'precision': 'int',
    '

In [4]:

image = ee.Image('LANDSAT/LC08/C02/T1_L2/LC08_008057_20241011')
rgb = image.select(['SR_B4', 'SR_B3', 'SR_B2'])

vis = {'min': 7000, 'max': 20000, 'gamma': 2.5}

# Puedes cambiar el tamaño aquí
Map = geemap.Map(height=300, width='70%')  # También puedes usar '600px' o '1000px'
Map.centerObject(image, zoom=8)
Map.addLayer(rgb, vis, 'Landsat 8 RGB')
Map

Map(center=[4.337462216486839, -74.48506744410943], controls=(WidgetControl(options=['position', 'transparent_…

In [ ]:
import ee
import geemap

# Inicializar Earth Engine
ee.Initialize()

# Crear un mapa interactivo
Map = geemap.Map()

# --------------------------------------------
# 1. Cargar tu GeoJSON
# --------------------------------------------
# Supongamos que tu archivo es 'zona_estudio.geojson'
import geopandas as gpd

gdf = gpd.read_file('zona_estudio.geojson')

# Convertir el GeoJSON a EE Feature
import json
with open('zona_estudio.geojson') as f:
    data = json.load(f)

zona_estudio = ee.FeatureCollection(data)

# Mostrar en el mapa
Map.centerObject(zona_estudio, 14)
Map.addLayer(zona_estudio, {"color": "00ffbf"}, "Zona de Estudio")

# --------------------------------------------
# 2. Filtrar imágenes Landsat 8 dentro de la zona y en 2023
# --------------------------------------------

landsat8 = ee.ImageCollection('LANDSAT/LC08/C02/T1_L2') \
    .filterBounds(zona_estudio) \
    .filterDate('2023-01-01', '2023-12-31')

print('Número de imágenes:', landsat8.size().getInfo())

# --------------------------------------------
# 3. Obtener imagen con menos nubes
# --------------------------------------------

image = landsat8.sort('CLOUD_COVER').first()
print('Propiedades de la imagen más limpia:')
print(image.getInfo())

# --------------------------------------------
# 4. Visualizar la imagen en RGB (True Color)
# --------------------------------------------

# Visualización básica
rgb_vis = {
    'bands': ['SR_B4', 'SR_B3', 'SR_B2'],
    'min': 7000,
    'max': 20000,
    'gamma': 2.5
}

Map.addLayer(image, rgb_vis, "True Color")

# --------------------------------------------
# 5. Aplicar escala para reflectancia
# --------------------------------------------

escala = 0.0000275
intercepto = -0.2

image_SR = image.select(['SR_B1', 'SR_B2', 'SR_B3', 'SR_B4', 'SR_B5', 'SR_B6', 'SR_B7']) \
    .multiply(escala).add(intercepto)

# Visualización de reflectancia
reflectance_vis = {
    'bands': ['SR_B4', 'SR_B3', 'SR_B2'],
    'min': 0.00,
    'max': 0.12,
    'gamma': 2.0
}

Map.addLayer(image_SR, reflectance_vis, "Reflectancia True Color")

# --------------------------------------------
# 6. Visualizaciones en Falso Color
# --------------------------------------------

# RGB 572
false_color_572 = {
    'bands': ['SR_B5', 'SR_B7', 'SR_B2'],
    'min': 0.00,
    'max': 0.35,
    'gamma': 1.5
}
Map.addLayer(image_SR, false_color_572, "Falso Color 572")

# RGB 564
false_color_564 = {
    'bands': ['SR_B5', 'SR_B6', 'SR_B4'],
    'min': 0.00,
    'max': 0.35,
    'gamma': 1.5
}
Map.addLayer(image_SR, false_color_564, "Falso Color 564")

# --------------------------------------------
# 7. Recortar la imagen al área de interés
# --------------------------------------------

image_clipped = image_SR.clip(zona_estudio)
Map.addLayer(image_clipped, false_color_564, "Recorte Falso Color 564")

# Mostrar el mapa
Map
